In [1]:
import numpy as np
import pandas as pd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from keras.layers import Dense, SimpleRNN

# Hybrid metadata creation

In [2]:
def wholeData(path_train,path_test):
    #Split the data
    Train = pd.read_csv(path_train)
    Test = pd.read_csv(path_test)
    
    X_train = Train.iloc[:,:-1]
    Y_train = Train.iloc[:,-1]
    
    X_test = Test.iloc[:,:-1]
    Y_test = Test.iloc[:,-1]
    
    X_entire = pd.concat([X_train,X_test])
    Y_entire = pd.concat([Y_train,Y_test])
    return X_entire,Y_entire

In [9]:
def tl(path_train, path_test):
    #Split the data
    Train = pd.read_csv(path_train)
    Test = pd.read_csv(path_test)
    
    X_train = Train.iloc[:,:-1]
    Y_train = Train.iloc[:,-1]

    X_test = Test.iloc[:,:-1]
    Y_test = Test.iloc[:,-1]

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)
        
    list_dnn = ["cle_dnn.h5", "hun_dnn.h5"]
    output = pd.DataFrame()
    for i in list_dnn:
        model = tf.keras.models.load_model('../Models/DNN_only/' + i, compile=False)
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        model2 = Sequential()
        model2.add(Dense(64, input_shape=(22,), activation='relu'))
        model2.add(Dense(128, activation='relu'))
        model2.add(Dense(256, activation='relu'))
        model2.add(Dense(16, activation='sigmoid'))

        model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        X_entire,labels = wholeData(path_train,path_test)

        Y_predictions = model2.predict(X_entire)
        Y_predictions.shape

        temp = pd.DataFrame(Y_predictions)
        output = pd.concat([output,temp],axis = 1)
        
    list_cnn = ["vir_cnn.h5"]
    for j in list_cnn:
        model = tf.keras.models.load_model('../Models/CNN_only/' + j, compile=False)
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        model2 = Sequential()
        model2.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(22,1)))
        model2.add(MaxPooling1D(pool_size=2))

        model2.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
        model2.add(MaxPooling1D(pool_size=2))

        model2.add(Flatten())
        model2.add(Dense(16, activation='relu'))

        model2.set_weights(model.get_weights()[:-2]) 

        model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        X_entire,labels = wholeData(path_train,path_test)

        Y_predictions = model2.predict(X_entire)
        Y_predictions.shape

        temp = pd.DataFrame(Y_predictions)
        output = pd.concat([output,temp],axis = 1)
    
    labels = labels.reset_index()
    labels = labels.drop(columns = ['index'])

    output = pd.concat([output,labels],axis = 1)
    
    return output

# Cleveland

In [10]:
path_train = '../TrainTestData/cle_train.csv'
path_test = '../TrainTestData/cle_test.csv'

output = tl(path_train, path_test)

output.to_csv("../Metadata Learning/" + "cle_metadata_hybrid.csv",index = False)

# Virginia

In [11]:
path_train = '../TrainTestData/vir_train.csv'
path_test = '../TrainTestData/vir_test.csv'

output = tl(path_train, path_test)

output.to_csv("../Metadata Learning/" + "vir_metadata_hybrid.csv",index = False)

# Hungarian

In [12]:
path_train = '../TrainTestData/hun_train.csv'
path_test = '../TrainTestData/hun_test.csv'

output = tl(path_train, path_test)

output.to_csv("../Metadata Learning/" + "hun_metadata_hybrid.csv",index = False)

# Switzerland

In [13]:
path_train = '../TrainTestData/swi_train.csv'
path_test = '../TrainTestData/swi_test.csv'

output = tl(path_train, path_test)

output.to_csv("../Metadata Learning/" + "swi_metadata_hybrid.csv",index = False)